In [4]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent  # volta de notebooks/ para a raiz (jarvis-planejador-financeiro)
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from database import init_database, get_connection

In [5]:
from database import init_database, get_connection

# Garante que o banco e as tabelas existam
init_database()

with get_connection() as conn:
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;").fetchall()

tables


In [6]:
from pprint import pprint

with get_connection() as conn:
    schema = {}
    for (name,) in tables:
        schema[name] = conn.execute(f"SELECT sql FROM sqlite_master WHERE type='table' AND name='{name}';").fetchone()[0]

pprint(schema)


{'categories': 'CREATE TABLE categories (\n'
               '                category_id INTEGER PRIMARY KEY '
               'AUTOINCREMENT,\n'
               '                user_phone TEXT,\n'
               '                category_name TEXT,\n'
               '                description TEXT,\n'
               '                created_at DATETIME DEFAULT '
               'CURRENT_TIMESTAMP,\n'
               '                FOREIGN KEY (user_phone) REFERENCES '
               'users(user_phone)\n'
               '            )',
 'sqlite_sequence': 'CREATE TABLE sqlite_sequence(name,seq)',
 'transactions': 'CREATE TABLE transactions (\n'
                 '                transaction_id INTEGER PRIMARY KEY '
                 'AUTOINCREMENT,\n'
                 '                user_phone TEXT,\n'
                 '                category_id INTEGER,\n'
                 '                amount REAL,\n'
                 '                expense_description TEXT,\n'
             